In [26]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('data.csv', index_col=0).reset_index().drop('index', axis=1)

In [3]:
df.head()

,name,brand,2G,3G,4G,5G,Announced,Status,Weight,Length,...,Bluetooth,GPU,Loudspeaker,3.5mm jack,Chipset,Network,Internal,Card slot,RAM,Storage
0,alcatel 1b (2022),alcatel,True,True,True,False,2022.0,Available,172.0,146.7,...,"5.0, A2DP",PowerVR GE8300,True,True,Mediatek MT6761 Helio A22 (12 nm),"GSM / HSPA / LTE HSPA 42.2/11.1 Mbps, LTE Cat4...",32GB 2GB RAM,microSDXC (dedicated slot),2.0,32.0
1,alcatel 1l pro (2021),alcatel,True,True,True,False,2021.0,Available,190.0,156.4,...,"4.2, A2DP",IMG8322,True,True,Unisoc SC9863A (28nm),"GSM / HSPA / LTE HSPA 21.1/5.76 Mbps, LTE Cat4...",32GB 2GB RAM,microSDXC (uses shared SIM slot),2.0,32.0
2,alcatel 1 (2021),alcatel,True,True,True,False,2021.0,Available,134.0,137.6,...,"4.2, A2DP, LE",PowerVR GE8100,True,True,Mediatek MT6739 (28 nm),"GSM / HSPA / LTE HSPA 42.2/11.5 Mbps, LTE Cat4...","8GB 1GB RAM, 16GB 1GB RAM",microSDHC (dedicated slot),1.0,8.0
3,alcatel 3l (2021),alcatel,True,True,True,False,2021.0,Available,194.0,165.6,...,"5.0, A2DP, LE",PowerVR GE8320,True,True,Mediatek MT6762D Helio P22 (12 nm),"GSM / HSPA / LTE HSPA 42.2/5.76 Mbps, LTE Cat4...",64GB 4GB RAM,microSDXC (only single sim model),4.0,64.0
4,alcatel 1s (2021),alcatel,True,True,True,False,2021.0,Available,190.0,165.6,...,"5.0, A2DP, LE",PowerVR GE8320,True,True,Mediatek MT6762D Helio P22 (12 nm),"GSM / HSPA / LTE HSPA 42.2/5.76 Mbps, LTE Cat4...",32GB 3GB RAM,microSDXC (uses shared SIM slot),3.0,32.0


In [4]:
df.columns

Index(['name', 'brand', '2G', '3G', '4G', '5G', 'Announced', 'Status',
       'Weight', 'Length', 'Width', 'Diameter', 'SIM', 'Display Type',
       'Display Size', 'ppi', 'body ratio', 'OS', 'battery_capacity', 'Price',
       'CPU', 'ratio', 'pixel', 'WLAN', 'Colors', 'Sensors', 'Bluetooth',
       'GPU', 'Loudspeaker', '3.5mm jack', 'Chipset', 'Network', 'Internal',
       'Card slot', 'RAM', 'Storage'],
      dtype='object')

In [5]:
df.drop(['name', '2G', 'Status', 'Card slot', 'Internal', 'Network', 'Colors', 'WLAN', 'Bluetooth', 'Sensors', 'GPU'], axis=1, inplace=True)

ستون اسم و توجی براساس چیزی که خودت گفته بودی حذفشون منطقیه
ستون استتوس بنظرم به تاریخ خیلی مرتبطه و معیاری برای قیمت گذاری نیست برا همین حذفش کردم
ستون کارد اسلات برای قیمت گداری چیز به درد بخوری نبود چون خیلی از گوشی های امروزی اصلا ندارن همچین جیزی و در گذشته شاید تاثیر گذار میبوده
ستون نتورک و ویلن همون سه نمیانده تریجی و فورجی و فایو جی کافیه اینا هم خیلی گستردن (باعث یجورایی گیجی مدل میشه) برای همین حذف کردم
ستون رنگ همون که خودت گفتی و ستون اینترنال هم جدا شده تو رم و استوریج داریمش
ستون بلوتوث چون ورژن و نوع های مختلفی داشت خیلی مدل رو گیج میکرد و همه گوشی ها بولوتوث داشتن
سنسور هم اولا لیستی بود و تقریبا به چیپست و برند وابسته است برای همین حذفش کردم و البته برای قیمت گذاری بنطرم خیلی چیپ باشه
گرافیگ هم با فرض اینکه به سی پی بو وابستگی داره و گستردگی زیادی داره حذفش کردم

In [6]:
df.columns

Index(['brand', '3G', '4G', '5G', 'Announced', 'Weight', 'Length', 'Width',
       'Diameter', 'SIM', 'Display Type', 'Display Size', 'ppi', 'body ratio',
       'OS', 'battery_capacity', 'Price', 'CPU', 'ratio', 'pixel',
       'Loudspeaker', '3.5mm jack', 'Chipset', 'RAM', 'Storage'],
      dtype='object')

In [7]:
df.isna().sum()

brand                 0
3G                    0
4G                    0
5G                    0
Announced             0
Weight                0
Length                0
Width                 0
Diameter              0
SIM                   0
Display Type          0
Display Size          0
ppi                   0
body ratio            0
OS                    0
battery_capacity      0
Price                 0
CPU                 279
ratio                 0
pixel                 0
Loudspeaker           0
3.5mm jack            0
Chipset             257
RAM                   0
Storage               0
dtype: int64

In [8]:
df['Announced'] = df['Announced'].astype('int')

In [9]:
def Dimensions(x):
    try:
        return float(x)
    except:
        if 'Unfolded' in x:
            return float(x.split(': ')[1])
        elif 'mm' in x:
            return float(x.split('mm')[0])
        elif '-' in x:
            return float(x.split('-')[1]) - float(x.split('-')[0])

In [10]:
df['Length'] = df['Length'].apply(Dimensions)
df['Length'] = df['Length'].astype('float')
df['Width'] = df['Width'].apply(Dimensions)
df['Width'] = df['Width'].astype('float')
df['Diameter'] = df['Diameter'].apply(Dimensions)
df['Diameter'] = df['Diameter'].astype('float')

In [11]:
df = pd.concat([df, pd.get_dummies(df['SIM'])], axis=1)
df.drop('SIM', axis=1, inplace=True)

In [12]:
def Display_Type(x):
    if 'IPS' in x:
        return 'IPS'
    if 'TFT' in x:
        return 'TFT'
    if 'Retina' in x:
        return 'Retina'
    if 'Super AMOLED' in x:
        return 'Super AMOLED'
    if 'AMOLED' in x:
        return 'AMOLED'
    if 'OLED' in x:
        return 'OLED'
    if 'LCD' in x:
        return 'LCD'
    if 'PLS' in x:
        return 'PLS'
    else:
        return 'LCD'

In [13]:
df['Display Type'] = df['Display Type'].apply(Display_Type)

In [14]:
def OS(x):
    if 'Android' in x:
        return 'Android'
    if 'iOS' in x or 'iPadOS' in x:
        return 'iOS'
    else:
        return 'Other'

In [15]:
df['OS'] = df['OS'].apply(OS)

In [16]:
ratio_1 = list()
ratio_2 = list()
for ax_1, ax_2 in df['ratio'].str.split(':'):
    ratio_1.append(float(ax_1))
    ratio_2.append(float(ax_2))
df['ratio_1'] = ratio_1
df['ratio_2'] = ratio_2
df.drop('ratio', axis=1, inplace=True)

In [17]:
def Chipset(x):
    if 'Qualcom' in str(x) or 'Snapdragon' in str(x):
        return 'Qualcom'
    if 'Mediatek' in str(x) or 'MediaTek' in str(x):
        return 'Mediatek'
    if 'Intel' in str(x):
        return 'Intel'
    if 'Nvidia' in str(x) or 'NVIDIA' in str(x):
        return 'Nvidia'
    if 'Apple' in str(x):
        return 'Apple'
    if 'Unisoc' in str(x) or 'Spreadtrum' in str(x):
        return 'Unisoc'
    if 'Exynos' in str(x) or 'Hummingbird' in str(x):
        return 'Exynos'
    if 'Kirin' in str(x) or 'Huawei' in str(x):
        return 'Kirin'
    else:
        return np.nan

In [18]:
df['Chipset'] = df['Chipset'].apply(Chipset)

In [19]:
knn_imputer = KNNImputer(n_neighbors=5)
df['CPU'] = knn_imputer.fit_transform(df[['CPU']])

براساس 5 همسایه نزدیک، ستون سی پی یو رو پر کردم

In [20]:
df.dropna(inplace=True)

نال های چیپست رو حذف کردم و راحت (برای ستون جیپست فقط برند هارو نگه داشتم)

In [21]:
df.select_dtypes('object').columns

Index(['brand', 'Display Type', 'OS', 'Chipset'], dtype='object')

In [22]:
df.to_csv('data_no_mising_value_cat_col.csv', index=False)

In [23]:
from sklearn.preprocessing import LabelEncoder
df['brand'] = LabelEncoder().fit_transform(df['brand'])
df['Display Type'] = LabelEncoder().fit_transform(df['Display Type'])
df['OS'] = LabelEncoder().fit_transform(df['OS'])
df['Chipset'] = LabelEncoder().fit_transform(df['Chipset'])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2348 entries, 0 to 2649
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   brand             2348 non-null   int32  
 1   3G                2348 non-null   bool   
 2   4G                2348 non-null   bool   
 3   5G                2348 non-null   bool   
 4   Announced         2348 non-null   int32  
 5   Weight            2348 non-null   float64
 6   Length            2348 non-null   float64
 7   Width             2348 non-null   float64
 8   Diameter          2348 non-null   float64
 9   Display Type      2348 non-null   int32  
 10  Display Size      2348 non-null   float64
 11  ppi               2348 non-null   float64
 12  body ratio        2348 non-null   float64
 13  OS                2348 non-null   int32  
 14  battery_capacity  2348 non-null   float64
 15  Price             2348 non-null   float64
 16  CPU               2348 non-null   float64
 17  

In [25]:
df.to_csv('data_no_mising_value_label_encoder_col.csv', index=False)